In [7]:
import pandas as pd
import zipfile
import os

# === Set up paths ===
base_path = "input_data1/"
output_path = "cleaned_data/"
os.makedirs(output_path, exist_ok=True)

# === Step 1: Unzip files ===
def unzip_file(zip_path, extract_to):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

unzip_file(base_path + "hmda_2007_nationwide_first-lien-owner-occupied-1-4-family-records_labels.zip", base_path + "2007_data/")
unzip_file(base_path + "hmda_2017_nationwide_first-lien-owner-occupied-1-4-family-records_labels (1).zip", base_path + "2017_data/")
unzip_file(base_path + "2023_combined_mlar.zip", base_path + "2023_data/")

# === Step 2: Chunked processing for 2007/2017 ===
def process_hmda_file(folder, output_filename, year, delimiter, state_codes):
    chunk_size = 100000
    first_chunk = True
    columns_to_keep = [
        'year', 'state_code', 'county_code', 'census_tract_number',
        'loan_amount_000s', 'applicant_income_000s',
        'property_value', 'loan_purpose', 'lien_status'
    ]

    for file in os.listdir(folder):
        if file.endswith(".csv") or file.endswith(".txt"):
            path = os.path.join(folder, file)
            break

    for chunk in pd.read_csv(path, sep=delimiter, chunksize=chunk_size, low_memory=False):
        chunk.columns = [col.strip().lower() for col in chunk.columns]
        if 'state_code' not in chunk.columns:
            continue
        chunk = chunk[chunk['state_code'].astype(str).isin(state_codes)]
        filters = (
            (chunk['loan_type'] == 1) if 'loan_type' in chunk.columns else True
        ) & (
            (chunk['property_type'] == 1) if 'property_type' in chunk.columns else True
        ) & (
            (chunk.get('owner_occupancy', chunk.get('occupancy')) == 1)
        ) & (
            (chunk['action_taken'] == 1) if 'action_taken' in chunk.columns else True
        )
        chunk = chunk[filters]
        tract_field = 'census_tract_number' if 'census_tract_number' in chunk.columns else 'census_tract'
        chunk = chunk.dropna(subset=[tract_field, 'loan_amount_000s', 'applicant_income_000s'])
        chunk['year'] = year
        chunk = chunk.rename(columns={tract_field: 'census_tract'})
        existing_cols = [col for col in columns_to_keep if col in chunk.columns]
        chunk = chunk[existing_cols]
        chunk.to_csv(os.path.join(output_path, output_filename), mode='a', index=False, header=first_chunk)
        first_chunk = False

    print(f"✅ Finished {year}: saved to cleaned_data/{output_filename}")

# === Step 3: Special handling for 2023 data ===
def process_2023_file(folder, output_filename):
    chunk_size = 100000
    first_chunk = True
    colnames = [
        "as_of_year", "lei", "loan_type", "property_type", "loan_purpose", "occupancy", 
        "preapproval", "loan_amount", "action_taken", "state_abbr", "county_code", "census_tract"
    ]
    for file in os.listdir(folder):
        if file.endswith(".csv") or file.endswith(".txt"):
            path = os.path.join(folder, file)
            break

    for chunk in pd.read_csv(path, sep="|", chunksize=chunk_size, header=None, low_memory=False):
        sub = chunk.iloc[:, :12].copy()
        sub.columns = colnames
        sub = sub[sub['state_abbr'].isin(['FL', 'TX'])]
        sub = sub[
            (sub['loan_type'] == 1) &
            (sub['property_type'] == 1) &
            (sub['occupancy'] == 1) &
            (sub['action_taken'] == 1)
        ]
        sub = sub.dropna(subset=['loan_amount', 'census_tract', 'county_code'])
        sub['year'] = 2023
        sub.rename(columns={'loan_amount': 'loan_amount_000s'}, inplace=True)
        sub = sub[['year', 'state_abbr', 'county_code', 'census_tract', 'loan_amount_000s']]
        sub.to_csv(os.path.join(output_path, output_filename), mode='a', index=False, header=first_chunk)
        first_chunk = False

    print(f"✅ Finished 2023: saved to cleaned_data/{output_filename}")

# === Step 4: Run All ===
state_codes = ['12', '48']
process_hmda_file(base_path + "2007_data/", "cleaned_2007_FL_TX.csv", 2007, ",", state_codes)
process_hmda_file(base_path + "2017_data/", "cleaned_2017_FL_TX.csv", 2017, ",", state_codes)
process_2023_file(base_path + "2023_data/", "cleaned_2023_FL_TX.csv")

print("🎉 All files processed and saved in cleaned_data/")

✅ Finished 2007: saved to cleaned_data/cleaned_2007_FL_TX.csv
✅ Finished 2017: saved to cleaned_data/cleaned_2017_FL_TX.csv
✅ Finished 2023: saved to cleaned_data/cleaned_2023_FL_TX.csv
🎉 All files processed and saved in cleaned_data/
